<a href="https://colab.research.google.com/github/shonendumm/pyspark_lessons/blob/main/Snowflake_Python_Connector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
# !pip install snowflake-connector-python

!pip install "snowflake-connector-python[pandas]"


In [31]:
import os

import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

import pandas as pd

# Read data from CSV

In [40]:
df = pd.DataFrame({"NAME":["Alex", "Ben", "Charles"], "AGE": [22, 33, 44]})

# Use upper case for the column names, because column names in tables will be in CAPS.
# Need to be consistent in order to add this df data to the tables.

In [41]:
df.head()

,NAME,AGE
0,Alex,22
1,Ben,33
2,Charles,44


# Write to Snowflake

In [34]:
conn = snowflake.connector.connect(
    user="____",
    password="____",
    account="_____",
    warehouse="TEST_WAREHOUSE",
    database="DEV",
    schema="CDM"
    )

cs = conn.cursor()

In [35]:
# Define the table creation SQL
table_creation_sql = '''
CREATE TABLE EXAMPLE_TABLE (
    NAME STRING,
    AGE INTEGER
);
'''

# Execute the SQL command to create the table
cs.execute(table_creation_sql)

In [42]:
# Write data to table

write_pandas(conn, df, table_name="EXAMPLE_TABLE")

(True,
 1,
 3,
 [('gljqcvvavm/file0.txt', 'LOADED', 3, 3, 1, 0, None, None, None, None)])

In [44]:
# Another way to write table and insert data:

conn.cursor().execute(
    "CREATE OR REPLACE TABLE "
    "test_table(col1 integer, col2 string)")

conn.cursor().execute(
    "INSERT INTO test_table(col1, col2) VALUES " +
    "    (123, 'test string1'), " +
    "    (456, 'test string2')")

# Read from Snowflake

In [52]:
cs.execute("SELECT * FROM DEV.CDM.EXAMPLE_TABLE")
rows = cs.fetchall()
for row in rows:
  print(row)

('Alex', 22)
('Ben', 33)
('Charles', 44)


# Dataframe from Snowflake table

In [54]:
cs.execute("select * from EXAMPLE_TABLE")
df_from_snow = cs.fetch_pandas_all()
df_from_snow

,NAME,AGE
0,Alex,22
1,Ben,33
2,Charles,44


In [55]:
cs.close()
conn.close()